In [18]:
import pandas as pd
from rank_bm25 import BM25Okapi
from tqdm.notebook import tqdm
import ast
import random

In [11]:
MedQA_path = "/scratch/s190619/MSMARCO/Data/" + "disorders_table_dev-test.csv"
df_medQA = pd.read_csv(MedQA_path, sep=',', encoding="utf8")
df_medQA.head()

,qid,query,answer,options,meta_info,Disorder,CUI,TUI,short_category,long_category,description,cui_METAMAP
0,0,A 5-year-old girl is brought to the emergency ...,Cyclic vomiting syndrome,"{'A': 'Cyclic vomiting syndrome', 'B': 'Gastro...",step2&3,True,NaN,NaN,NaN,NaN,NaN,['C0152164']
1,1,A 19-year-old boy presents with confusion and ...,Hypoperfusion,"{'A': 'Hypoperfusion', 'B': 'Hyperglycemia', '...",step1,True,C0442856,T046,DISO,Disorders,Pathologic Function,['C0442856']
2,2,A 41-year-old woman presents to her primary ca...,Iron deficiency,"{'A': 'Vitamin B12 deficiency', 'B': 'Folate d...",step2&3,True,C0240066,T047,DISO,Disorders,Disease or Syndrome,['C0240066']
3,3,A 56-year-old man with known coronary artery d...,Monomorphic ventricular tachycardia,"{'A': 'Premature ventricular contractions', 'B...",step1,True,NaN,NaN,NaN,NaN,NaN,['C0344431']
4,4,A 16-year-old female high school student is br...,Oppositional defiant disorder,"{'A': 'Reactive attachment disorder', 'B': 'Co...",step2&3,True,C0029121,T048,DISO,Disorders,Mental or Behavioral Dysfunction,['C0029121']


In [12]:
path = "/scratch/s190619/Data_etc/FiD/Reader_Results/Model2_msmarco_medical_title2/final_output.txt"

In [13]:
df = pd.read_csv(path,sep="\t",header=None)
df.columns = ["qid","FiD_result"]
df.head()

,qid,FiD_result
0,0,Vomitor syndrome
1,1,Hyperparathyroidism
2,2,Iron deficiency
3,3,Hypertension
4,4,Psychiatric illness


In [14]:
for i in range(10):
    print(df["FiD_result"].iloc[i])
    print(df_medQA["answer"].iloc[i])
    print(df_medQA["options"].iloc[i])
    print("\n")

Vomitor syndrome
Cyclic vomiting syndrome
{'A': 'Cyclic vomiting syndrome', 'B': 'Gastroenteritis', 'C': 'Hypertrophic pyloric stenosis', 'D': 'Gastroesophageal reflux disease', 'E': 'Acute intermittent porphyria'}


Hyperparathyroidism
Hypoperfusion
{'A': 'Hypoperfusion', 'B': 'Hyperglycemia', 'C': 'Metabolic acidosis', 'D': 'Hypokalemia', 'E': 'Hypophosphatemia'}


Iron deficiency
Iron deficiency
{'A': 'Vitamin B12 deficiency', 'B': 'Folate deficiency', 'C': 'Iron deficiency', 'D': 'Infiltrative bone marrow process', 'E': 'Intravascular hemolysis'}


Hypertension
Monomorphic ventricular tachycardia
{'A': 'Premature ventricular contractions', 'B': 'Monomorphic ventricular tachycardia', 'C': 'Mitral regurgitation', 'D': 'Third-degree heart block', 'E': 'Acute pericarditis'}


Psychiatric illness
Oppositional defiant disorder
{'A': 'Reactive attachment disorder', 'B': 'Conduct disorder', 'C': 'Antisocial personality disorder', 'D': 'Attention-deficit hyperactivity disorder', 'E': 'Oppos

In [15]:
def BM25_MC(options, FiD_result):
    
    opts = list(ast.literal_eval(options).values())
    tokenized_options = [doc.lower().replace("\n","").split(" ") for doc in opts]
    bm25 = BM25Okapi(tokenized_options)
    tokenized_FiD = FiD_result.split(" ")
    doc_scores = bm25.get_scores(tokenized_FiD)
    relevant_passages = bm25.get_top_n(tokenized_FiD, opts, n=5)
    return relevant_passages

In [21]:
print_max = 10
printed = 0
correct = 0
correct_MC = 0

for i in range(df.shape[0]):
    FiD_result = df["FiD_result"].iloc[i]
    answer = df_medQA["answer"].iloc[i]
    if FiD_result == answer:
        correct += 1
    BM25_rank = BM25_MC(df_medQA["options"].iloc[i],FiD_result)
    if BM25_rank[0] == answer:
        correct_MC += 1
    
    if printed < print_max:
        print_rand = random.uniform(0, 1)
        if print_rand > 0.98:
            print(FiD_result)
            print(BM25_rank)
            print(answer)
            print(df_medQA["options"].iloc[i])
            print("\n")
            printed += 1

HELLP syndrome
['Viral meningitis', 'Idiopathic intracranial hypertension', 'Subarachnoid hemorrhage', 'Hypertensive emergency', 'Cerebral venous thrombosis']
Cerebral venous thrombosis
{'A': 'Cerebral venous thrombosis', 'B': 'Hypertensive emergency', 'C': 'Subarachnoid hemorrhage', 'D': 'Idiopathic intracranial hypertension', 'E': 'Viral meningitis'}


Atrial septal defect
['Atrial septal defect', 'Coarctation of the aorta\n"', 'Pulmonary valve stenosis', 'Benign heart murmur', 'Patent ductus arteriosus']
Atrial septal defect
{'A': 'Patent ductus arteriosus', 'B': 'Benign heart murmur', 'C': 'Atrial septal defect', 'D': 'Pulmonary valve stenosis', 'E': 'Coarctation of the aorta\n"'}


HELP
['Serial hCG', 'Abdominal ultrasound', 'Transvaginal ultrasound', 'Methotrexate', 'Surgery']
Surgery
{'A': 'Surgery', 'B': 'Methotrexate', 'C': 'Transvaginal ultrasound', 'D': 'Abdominal ultrasound', 'E': 'Serial hCG'}


Yeast cell carcinoma
['Increased production of prolactin', 'Increased aromatas

In [23]:
correct_MC/(df.shape[0]-correct)

0.2159709618874773

In [25]:
correct/df.shape[0]

0.09819967266775777

In [27]:
import json
path = "/scratch/s190619/MSMARCO/Data/FindZebra/FiD_reader_data/Model3.json"

with open(path) as json_file:
    data_test = json.load(json_file)

In [28]:
data_test[0]

{'question': 'A 5-year-old girl is brought to the emergency department by her mother because of multiple episodes of nausea and vomiting that last about 2 hours. During this period, she has had 6–8 episodes of bilious vomiting and abdominal pain. The vomiting was preceded by fatigue. The girl feels well between these episodes. She has missed several days of school and has been hospitalized 2 times during the past 6 months for dehydration due to similar episodes of vomiting and nausea. The patient has lived with her mother since her parents divorced 8 months ago. Her immunizations are up-to-date. She is at the 60th percentile for height and 30th percentile for weight. She appears emaciated. Her temperature is 36.8°C (98.8°F), pulse is 99/min, and blood pressure is 82/52 mm Hg. Examination shows dry mucous membranes. The lungs are clear to auscultation. Abdominal examination shows a soft abdomen with mild diffuse tenderness with no guarding or rebound. The remainder of the physical exami